In [1]:
import pandas
from google.cloud import bigquery
from google.oauth2 import service_account
#from params.py import PROJECT_ID, TABLE_ID
import sys; sys.path.append('..')
from params import PROJECT_ID, TABLE_ID
import os
import mediapipe as mp
import cv2
import matplotlib.pyplot as plt
import pandas as pd



image = '../raw_data/a/374f6c92-0cfb-11ee-876b-ce8ebbc91401.jpg'

# mediapiper l'image pour reconnaitre la main et extraire les points


def get_coordinates(image):
    """
    Calcul des coordonnées des points de la main et stockage dans un dataframe
    """
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

    data = []
    

    img = cv2.imread(image)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    results = hands.process(img_rgb)

    coords={}
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            wrist_x = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x
            wrist_y = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y
            wrist_z = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].z

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                z = hand_landmarks.landmark[i].z

                coords[f"x_{i}"]= x - wrist_x
                coords[f"y_{i}"]= y - wrist_y
                coords[f"z_{i}"]= z - wrist_z

        data.append(coords)


    df = pd.DataFrame(data)
    return df



# convertir en df identifuqe à data_ollection
# preproc la ligne X




In [4]:
df = get_coordinates(image)
df

,x_0,y_0,z_0,x_1,y_1,z_1,x_2,y_2,z_2,x_3,...,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20
0,0.0,0.0,0.0,0.061126,-0.031811,-0.01207,0.108672,-0.106618,-0.013327,0.131206,...,-0.011458,-0.003595,-0.242931,-0.031011,-0.001266,-0.173567,-0.01978,-0.007156,-0.142169,-0.00514


In [5]:
df = df.drop(['x_0','y_0','z_0'], axis=1)
df

,x_1,y_1,z_1,x_2,y_2,z_2,x_3,y_3,z_3,x_4,...,z_17,x_18,y_18,z_18,x_19,y_19,z_19,x_20,y_20,z_20
0,0.061126,-0.031811,-0.01207,0.108672,-0.106618,-0.013327,0.131206,-0.190649,-0.014928,0.121818,...,-0.011458,-0.003595,-0.242931,-0.031011,-0.001266,-0.173567,-0.01978,-0.007156,-0.142169,-0.00514
